In [1]:
#!pip install pandas pyarrow ydata-profiling ipywidgets


In [ ]:
import pandas as pd
import ipywidgets as widgets
from IPython.display import display
from ydata_profiling import ProfileReport
import webbrowser
import io
import time

# Widget para upload de arquivo .parquet
file_picker = widgets.FileUpload(
    accept='.parquet',
    multiple=False
)
display(file_picker)

def process_file(change):
    if not file_picker.value:
        print("❌ Nenhum arquivo selecionado.")
        return

    # ✅ Compatível com ipywidgets 8.x: value é uma TUPLA
    uploaded_file = file_picker.value[0]
    filename = uploaded_file['name']
    content = uploaded_file['content']

    print(f"📂 Lendo arquivo '{filename}'...")
    start = time.time()
    df = pd.read_parquet(io.BytesIO(content))
    print(f"✅ Arquivo lido com {df.shape[0]:,} linhas e {df.shape[1]} colunas.")

    # 🔢 Selecionar colunas numéricas
    num_cols = df.select_dtypes(include='number').columns.tolist()
    print(f"🔢 Colunas numéricas selecionadas: {len(num_cols)}")

    # 📉 Amostragem
    sample_size = min(10000, len(df))
    df_sample = df[num_cols].head(sample_size)
    
    print(f"📏 Linhas usadas: {sample_size}")

    # 🧠 Gerar relatório
    print("⚙️ Gerando relatório...")
    profile = ProfileReport(df_sample, title="Relatório Interativo", explorative=True)
    output_file = "../reports/relatorio_ydata.html"
    profile.to_file(output_file)
    
    elapsed = time.time() - start
    print(f"✅ Relatório salvo como: {output_file}")
    print(f"⏱️ Tempo total: {elapsed:.2f} segundos")

    # 🌐 Abrir no navegador
    webbrowser.open(output_file)

file_picker.observe(process_file, names='value')
